<a href="https://colab.research.google.com/github/xghan99/dsa4212-assignments/blob/main/GeneratingTrainingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import pandas as pd
df = pd.read_csv("train.csv")

In [ ]:
import numpy as np
import re
#Take 20000 data samples, we would be using the highlight column as a corpus
sample = np.random.randint(0,df.shape[0],20000)
df_sample = df.loc[sample,]
#Remove digits
df_sample['highlights'] = df_sample['highlights'].map(lambda x: re.sub(r'[0-9]', '', x))
#Lower Case
df_sample['highlights'] = df_sample['highlights'].map(lambda x: x.lower())
#Remove Next Line Characters
df_sample['highlights'] = df_sample['highlights'].map(lambda x: x.replace('\n',''))
#Remove stopwords
#df_sample['highlights'] = df_sample['highlights'].map(lambda x: rm_stopwords(x))
#Lemmatize text (this takes a while)
#df_sample['highlights'] = df_sample['highlights'].map(lambda x: nlp(x))
#df_sample['highlights'] = df_sample['highlights'].map(lambda x: " ".join([token.lemma_ for token in x]))

In [ ]:
df_sample = df_sample.reset_index()

In [ ]:
df_sample.shape

(20000, 4)

In [ ]:
df_sample.head()

,index,id,article,highlights
0,81123,e5e33fe1d102cf51b9ad5bc538d5c9b3c628edd2,Of the many World Cup squads already announced...,cesare prandelli named -man squad following - ...
1,201992,918106b6bf94a721f81b9c2f470a296d002561af,By . Daily Mail Reporter . PUBLISHED: . 10:00 ...,"nicholas john miller, , hijacked the bus after..."
2,240491,c35583eda74be5efce5bc2f26befc162337eea4d,"By . Olivia Williams . PUBLISHED: . 03:08 EST,...",gps tracker inside trailer led police straight...
3,80134,e31d15d4a0c530949f0b4f2f9c89f839715adf06,"By . Rex Gowar, Reuters . Radamel Falcao, Colo...",monaco striker was rumoured to travel to bogot...
4,32958,5d9e609a8cb14c9e68d8dfa40760429f5ab4c97e,An American explorer's claim to have found the...,u.n. experts say a wreck found off haiti is no...


In [ ]:
df_sample['no_punctuation'] = df_sample['highlights'].str.replace('[^\w\s]','')

<ipython-input-8-dc5f0d4f18ea>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_sample['no_punctuation'] = df_sample['highlights'].str.replace('[^\w\s]','')


In [ ]:
df_sample.head()

,index,id,article,highlights,no_punctuation
0,81123,e5e33fe1d102cf51b9ad5bc538d5c9b3c628edd2,Of the many World Cup squads already announced...,cesare prandelli named -man squad following - ...,cesare prandelli named man squad following dr...
1,201992,918106b6bf94a721f81b9c2f470a296d002561af,By . Daily Mail Reporter . PUBLISHED: . 10:00 ...,"nicholas john miller, , hijacked the bus after...",nicholas john miller hijacked the bus after a...
2,240491,c35583eda74be5efce5bc2f26befc162337eea4d,"By . Olivia Williams . PUBLISHED: . 03:08 EST,...",gps tracker inside trailer led police straight...,gps tracker inside trailer led police straight...
3,80134,e31d15d4a0c530949f0b4f2f9c89f839715adf06,"By . Rex Gowar, Reuters . Radamel Falcao, Colo...",monaco striker was rumoured to travel to bogot...,monaco striker was rumoured to travel to bogot...
4,32958,5d9e609a8cb14c9e68d8dfa40760429f5ab4c97e,An American explorer's claim to have found the...,u.n. experts say a wreck found off haiti is no...,un experts say a wreck found off haiti is not ...


In [ ]:
#Check Missing Data
df_sample.isna().sum()

index             0
id                0
article           0
highlights        0
no_punctuation    0
dtype: int64

In [ ]:
most_freq_words = pd.Series(' '.join(df_sample['no_punctuation']).lower().split()).value_counts()[:8500]

In [ ]:
most_freq_words = list(most_freq_words.index)

In [ ]:
most_freq_words

In [ ]:
from tqdm import tqdm
idx_2_word = {}
word_2_idx = {}
for i in tqdm(range(8500)):
  idx_2_word[i] = most_freq_words[i]
  word_2_idx[most_freq_words[i]] = i

100%|██████████| 8500/8500 [00:00<00:00, 970348.77it/s]


In [ ]:
idx_2_word[0]

'the'

In [ ]:
word_2_idx['the']

0

In [ ]:
import pickle
#Save the lookup tables
with open('idx_2_word.pkl', 'wb') as fp:
    pickle.dump(idx_2_word, fp)
    print('dictionary saved successfully to file')
with open('word_2_idx.pkl', 'wb') as fp:
    pickle.dump(word_2_idx, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file
dictionary saved successfully to file


In [ ]:
import string
def one_hot_map(doc):
  x = []
  for sentence in doc.split('.'):
    y = []
    for word in sentence.split():
      word = word.translate(str.maketrans('','',string.punctuation))
      if word in word_2_idx:
        y.append(word_2_idx[word])
      else:
        y.append(9000)
    x.append(y)
  return x
#Convert words in the sentence to numbers
df_sample['highlights_encoded'] = df_sample['highlights'].map(lambda x: one_hot_map(x))

In [ ]:
df_sample['highlights_encoded']

0        [[9000, 9000, 435, 83, 1065, 297, 9000, 980, 1...
1        [[4086, 251, 1845, 9000, 7217, 0, 873, 21, 4, ...
2        [[3723, 9000, 654, 5177, 563, 39, 1691, 1, 900...
3        [[3393, 418, 6, 6160, 1, 747, 1, 9000, 1, 3159...
4        [[6146], [4989], [385, 51, 4, 4561, 52, 84, 35...
                               ...                        
19995    [[4427, 9000, 9000, 5, 9000, 13, 2855, 11, 900...
19996    [[9000, 9000, 9000, 63, 8, 0, 767, 7, 9000], [...
19997    [[1296, 2243, 14, 853, 0, 9000, 24, 583, 3449,...
19998    [[406, 3590, 9, 5459, 103, 19, 1349, 3, 9000, ...
19999    [[9000, 387, 5835, 7253, 344, 1, 2235, 2, 7280...
Name: highlights_encoded, Length: 20000, dtype: object

In [ ]:
idx_2_word[435]

'named'

In [ ]:
df_sample['highlights']

0        cesare prandelli named -man squad following - ...
1        nicholas john miller, , hijacked the bus after...
2        gps tracker inside trailer led police straight...
3        monaco striker was rumoured to travel to bogot...
4        u.n. experts say a wreck found off haiti is no...
                               ...                        
19995    barry levinson shaves and brushes his teeth wi...
19996    four-bedroom nottinghamshire eco home on the m...
19997    tony abbott has confirmed the adf are training...
19998    bill gates is stepping down as chairman of mic...
19999    joran van der sloot plans to marry in peruvian...
Name: highlights, Length: 20000, dtype: object

In [ ]:
window = 2
#Create training data using skip-gram
from tqdm import tqdm
def create_training_data(doc):
  training_data = np.empty((0,2))
  for sentence in doc:
    sent_len = len(sentence)
    for i, word in enumerate(sentence):
      w_context = []
      w_target = sentence[i]
      if w_target == 0:
        continue
      for j in range(i-window, i+window+1):
        if j != i and j <= sent_len - 1 and j >= 0:
          w_context = sentence[j]
          if w_context == 0:
            continue
          training_data = np.append(training_data,[[w_target,w_context]],axis = 0)
  return training_data

In [ ]:
window = 2
#Create training data using CBOW
from tqdm import tqdm
def create_training_data(doc):
  training_data = np.empty((0,2))
  for sentence in doc:
    sent_len = len(sentence)
    for i, word in enumerate(sentence):
      w_input= []
      w_output = sentence[i]
      if w_output == 9000:
        continue
      for j in range(i-window, i+window+1):
        if j != i and j <= sent_len - 1 and j >= 0:
          w_input = sentence[j]
          if w_input == 9000:
            continue
          training_data = np.append(training_data,[[w_input,w_output]],axis = 0)
  return training_data

In [ ]:
tqdm.pandas()
df_sample['training_data'] = df_sample['highlights_encoded'].progress_map(lambda x: create_training_data(x))

100%|██████████| 20000/20000 [00:14<00:00, 1333.67it/s]


In [ ]:
df_sample['training_data']

0        [[83.0, 435.0], [1065.0, 435.0], [435.0, 83.0]...
1        [[251.0, 4086.0], [1845.0, 4086.0], [4086.0, 2...
2        [[654.0, 3723.0], [3723.0, 654.0], [5177.0, 65...
3        [[418.0, 3393.0], [6.0, 3393.0], [3393.0, 418....
4        [[51.0, 385.0], [4.0, 385.0], [385.0, 51.0], [...
                               ...                        
19995    [[13.0, 5.0], [5.0, 13.0], [2855.0, 13.0], [11...
19996    [[8.0, 63.0], [0.0, 63.0], [63.0, 8.0], [0.0, ...
19997    [[2243.0, 1296.0], [14.0, 1296.0], [1296.0, 22...
19998    [[3590.0, 406.0], [9.0, 406.0], [406.0, 3590.0...
19999    [[5835.0, 387.0], [7253.0, 387.0], [387.0, 583...
Name: training_data, Length: 20000, dtype: object

In [ ]:
df_sample['highlights_encoded']

0        [[9000, 9000, 435, 83, 1065, 297, 9000, 980, 1...
1        [[4086, 251, 1845, 9000, 7217, 0, 873, 21, 4, ...
2        [[3723, 9000, 654, 5177, 563, 39, 1691, 1, 900...
3        [[3393, 418, 6, 6160, 1, 747, 1, 9000, 1, 3159...
4        [[6146], [4989], [385, 51, 4, 4561, 52, 84, 35...
                               ...                        
19995    [[4427, 9000, 9000, 5, 9000, 13, 2855, 11, 900...
19996    [[9000, 9000, 9000, 63, 8, 0, 767, 7, 9000], [...
19997    [[1296, 2243, 14, 853, 0, 9000, 24, 583, 3449,...
19998    [[406, 3590, 9, 5459, 103, 19, 1349, 3, 9000, ...
19999    [[9000, 387, 5835, 7253, 344, 1, 2235, 2, 7280...
Name: highlights_encoded, Length: 20000, dtype: object

In [ ]:
df_sample['data_count'] = df_sample['training_data'].map(lambda x: len(x))

In [ ]:
df_sample['data_count'].sum()

2656408

In [ ]:
df_sample.to_pickle('df_sample.pkl')